# Steam Recommender System Project Overview

## 1. Introduction

- Steam is a video game digital distribution service. It has a marketplace where users can purchase games as well as several community features such as friends lists, groups, cloud storage, and in-game voice and chat functionality.
- In 2019 Steam had over 34,000 games with over 95 million monthly active users and in 2017 (last year available) purchases through Steam totaled $4.3 billion US dollars.
- My project is to train a recommender system for Steam.

### Recommender Systems Summary

- A **recommender system** is a model that seeks to predict the preferences a user would give to a collection of items.
- Two major classes: content-based filtering and collaborative filtering, although novel hybrid methods are also being developed.
- **Content-based filtering** create "profiles" of different classes of items and recommending users items from similar profiles to items they have already rated. 
    - Often implemented via k-means clustering, bayesian classifiers, decision trees, and artificial neural networks
    - Pros: easier to train, can make recommendations with relatively little info on the user
    - Cons: "echo chamber" effect, hard to implement across content types
- **Collaborative filtering** identify similar users based upon known common ratings, then recommend items to a target user based on what other similar users have rated highly.
    - Often implemented via nearest-neighbor algorithms or by matrix factorization models.
    - Pros: does not rely on content features, can learn preferences soley from user interactions with content
    - Cons: "cold start problem", scalability, sparsity
    
For this project we will implement a collaborative filtering model based on a matrix factorization model.

## 2.  Data Pre-Processing

- Source: https://steam.internet.byu.edu/

- Collected by researchers at BYU in 2013

- ~175GB .sql file that createa a database containing our data and several other tables

Since the file too large to load onto a local sql server, so first need to figure out how to extract the data we need. For this I split the file into 170 plain text files of size ~1GB and examined the conents. I skimmed them until I found those that contained the table we want. Here's a portion of one:

In [2]:
path = "/Volumes/Samsung_T5/Data/Games_Txt/seg_a.txt"

with open(path, "r") as f:
    text = f.read()
text[:1000]

"INSERT INTO `Games_1` VALUES (76561198015117560,41500,NULL,1785,'2013-06-25 08:26:33'),(76561198015117560,400,NULL,5,'2013-06-25 08:26:33'),(76561198015117470,8600,NULL,5,'2013-06-25 08:26:22'),(76561198015117470,8660,NULL,1,'2013-06-25 08:26:22'),(76561198015117430,41500,NULL,1708,'2013-06-25 08:26:18'),(76561198015117430,12210,NULL,969,'2013-06-25 08:26:18'),(76561198015117430,550,NULL,6696,'2013-06-25 08:26:18'),(76561198015117430,223530,NULL,NULL,'2013-06-25 08:26:18'),(76561198015117430,3590,NULL,7385,'2013-06-25 08:26:18'),(76561198015117430,400,NULL,494,'2013-06-25 08:26:18'),(76561198015117430,40930,NULL,61,'2013-06-25 08:26:18'),(76561198015117430,6120,NULL,227,'2013-06-25 08:26:18'),(76561198015117430,340,NULL,NULL,'2013-06-25 08:26:18'),(76561198015117430,380,NULL,NULL,'2013-06-25 08:26:18'),(76561198015117430,420,NULL,NULL,'2013-06-25 08:26:18'),(76561198015117430,440,NULL,312,'2013-06-25 08:26:18'),(76561198015117430,220,NULL,99,'2013-06-25 08:26:18'),(76561198015117430,6

Looking at the documentation on the source website, I see that the Games_1 table that we want is organized as follows:

- steamid: The steam ID of the user in question
- appid: The ID of a given app in the user's library
- playtime_2weeks: The total time the user has run this app in the two-week period leading up to when this data was requested from the API. Values are given in minutes.
- playtime_forever: The total time the user has run this app since adding it to their library. Values are given in minutes.
- dateretrieved: Timestamp of the time when this game data was requested from the API

So we can pull the data directly from the .txt files, which are much more manageable. For this project, I only care whether the user bought a game or not, so I just need to store Steam_id and App_id pairs.

In [3]:
import numpy as np
import pandas as pd

def clean_text(path):
    """
    Input: path to .txt file containing the data
    Output: list of lists, each row containing a data point of the form: [Steam Id, App Id, 1]
    """
    # Read File
    with open(path, "r") as f:
        text = f.read()

    # Delete SQL commands
    text = text.replace("INSERT INTO `Games_1` VALUES ", '')

    # Split by "("
    b = "),"
    lines = [w + ')' for w in text.split(b) if w]

    # Split each line into it's elements
    lines = [l.replace("(", "").split(",") for l in lines]

    # Only keep elements we want: steam_id, app_id, and a 1 which will make some next steps easier
    lines = np.array([ [np.int64(l[0]), np.int64(l[1]), 1] for l in lines])

    return lines

In [4]:
lines = clean_text(path)
lines[:10]

array([[76561198015117560,             41500,                 1],
       [76561198015117560,               400,                 1],
       [76561198015117470,              8600,                 1],
       [76561198015117470,              8660,                 1],
       [76561198015117430,             41500,                 1],
       [76561198015117430,             12210,                 1],
       [76561198015117430,               550,                 1],
       [76561198015117430,            223530,                 1],
       [76561198015117430,              3590,                 1],
       [76561198015117430,               400,                 1]])

For understanding our model, it is better to think of this data as entries in a (number of users) $\times$ (number of games) ratings martix $R$ where $R_{ij}$ is 1 if user $i$ would buy game $j$ and is zero otherwise. We can do this by pivoting the data above. In addition, I drop any users who have purchased <10 games since they are hard to make predictions for and because we already have more than enough data.

In [5]:
def data_to_sparse(data):
    """
    Inputs: list of data shaped like output of clean_text
    Output: pivoted dataframe, drops users with <10 games rated
    """
    df = pd.DataFrame(data, columns=['steam_id', 'app_id', 'interact'])

    # Drop users that interact with < 10 games

    grouped = df.groupby('steam_id').aggregate(np.count_nonzero)
    keep_ids = grouped[grouped['app_id'] >= 10].index
    df = df[df['steam_id'].isin(keep_ids)]

    # Create Pivot table

    pivot = pd.pivot_table(data= df, values='interact', index='steam_id', columns='app_id')

    pivot = pivot.astype(pd.SparseDtype("float", np.nan))

    return df, pivot

In [6]:
data, sparse_pivot = data_to_sparse(lines)

print(sparse_pivot.shape)
print('Percent Non-Nan: ',100*np.sum(sparse_pivot.count())/(sparse_pivot.shape[0]*sparse_pivot.shape[1]),'%')
sparse_pivot.head(5)

(127313, 2296)
Percent Non-Nan:  1.7427787988072196 %


app_id,10,20,30,40,50,60,70,80,100,130,...,238130,238170,238210,238530,238630,238710,238890,238930,240600,242110
steam_id,,,,,,,,,,,,,,,,,,,,,
76561197998360190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561197998360230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561197998360500,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561197998360540,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561197998360930,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Latent Factors/ Matrix Factorization Model

Let $R$ be an $n\times m$ ratings matrix, so that $R_{ij}$ denotes the rating user $i$ gave to item $j$. In our case $R$ is 0-1 matrix since our "ratings" are binary purchased or not purchased. We may assume that $R$ is large (hundreds of thousands or millions or users and items) and sparse (majority of entries are missing or 0). 

**Assumption**: every item has a set of attributes, called latent factors, which are shared across all items. Items differ in the degree to which they express these attributes. Additionally, every user has preferences, how much they like or dislike a particular attribute.

**Idea**: let $v_j$ be the attribute vector of item $j$ and let $u_i$ be the preference vector of user $i$. Then 
$$R_{ij} \approx u_i^Tv_j$$

**Goal**: let $k << n,m$. Then we want to find a $m \times k$ matrix $V$ and a $n \times k$ matrix $U$ such that
$$R \approx UV^T $$

Note that the rows of $V$ are the item attribute vectors and the rows of $U$ are the user preference vectors.

### 3.1 Choosing a Loss Function

###  Mean Squared Error:

The obvious choice, in spirit of this class, is to take the mean squared error over all the entries in $R$. Then our loss is:
$$ \ell(U,V) = \frac{1}{nm} \sum_{ij} ||R_{ij} - U_i V_j^T||^2. $$
This is equivalent to trying the minimize the Frobenius norm between $R$ and $U^TV$. Note that this can be solved by Singular Value Decomposition, where we keep only the $k$ largest singular values.

**Issues**: due to sparsity the SVD of the matrix will be near zero, which may lead to poor generalization performance. In particular, it will bad at predicting new/ unseen ratings since it will try to force all unseen entries to 0. 

### Mean-Squared-Error over Observed:

To avoid this, we could just take the mean squared error over the observed values. Let $\Omega$ be the observed set. Then:
$$ \ell(U,V) =\frac{1}{|\Omega|} \sum_{ij \, \in \Omega} ||R_{ij} - U_i V_j^T||^2. $$

**Issues**: Note that a matrix of all ones would have zero loss but is obviously not what we want. We need to find someone to limit the number of items the model tries to recommend to each user.

### Weighted Matrix Factorization Loss

The solution is to take the mean squared error over the observed values and add a ridge regularization term forcing $U,V$ to stay small to combat the issues above. Explicitly, we have:

$$\ell(U,V) = \frac{1}{|\Omega|} \sum_{ij \, \in \Omega} ||R_{ij} - U_i V_j^T||^2 + \lambda \bigg( \sum_{i} ||U_i||^2 +  \sum_j ||V_j||^2 \bigg)$$

where $\lambda \geq 0$ is a tuning parameter. In general, increase $\lambda$ if your data is more sparse as this pushes more non-observed values to zero. Note that if $\lambda =0$ then this is exactly the same as taking the mean squared error over just the observed entries as above.

### 3.2 Minimizing the Loss Function: Alternating Least Squares
 
There are two common approaches for minimizing the loss: Stochastic Gradient Descent (SGD) and Alternating Least Squares (ALS). SGD is very flexible and is a popular choice for minimizing many loss functions across many applications. However, by exploiting the special form our loss function takes we can actually minimize the loss much faster using the ALS method which we explain below.

We begin by initializing $U$ and $V$ randomly. Now treat $V$ as if it fixed so we are only optimizing over $U$. Then we can rewrite our loss as:

$$\ell(U) 
= \frac{1}{|\Omega|} \sum_{ij \, \in \Omega} ||R_{ij} - U_i V_j^T||^2 + \lambda \bigg( \sum_{i} ||U_i||^2 +  \sum_j ||V_j||^2 \bigg)
= \bigg( \sum_i \big(\frac{1}{|\Omega|} \sum_{j: ij \, \in \Omega} ||R_{j} - U_i V_j^T||^2 \big) + \lambda||U_i||^2 \bigg) +\lambda \sum_j ||V_j||^2.
$$

Note that each term in the sum is dependent on a single row of $U$, so we can minimize the loss by minimizing each  term individually. But each term is just an ordinary least squares problem with ridge regularization, which we can solve exactly in polynomial time! Thus we solve these problems for each row of $U$ and update $U$ accordingly.

After updating we alernate. Now we treat $U$ as fixed and repeat the process for $V$. Since at each step we are *guaranteed* to decrease the loss, which is not true for SGD, this process is guaranteed to converge and often does so much faster than SGD, usually using only 5-10 iterations.

## End

Please continue onto the "Data Exploration" notebook see some exploratory data analysis of our dataset, or go straight to the "Tuning and Evaluation" notebook to see how we implement this model using PySpark and how it performs on our dataset.